# Virtual Assistant

Connecting your chatbot to some source of external data source allows it to be more useful.
For example, linking to a rest API or database allows it to send queries.

The course discusses good practice in allowing users to amend query parameters. `sqllite3`
package is used and the course emphasises the safeguards that the `cursor.execute('SQL string ?', ?params)`
function has, preventing the users to send malicious queries that delete the database etc - SQL injection.

It is discouraged to use f strings or `.format()` for these same reasons.


In [1]:
import pandas as pd
from pyprojroot import here
import os

## Create a Database

In [2]:
from pathlib import Path
import sqlite3 # included in standard library

dbPath = os.path.join(here(), "data", "restaurants.db")
try:
    Path(dbPath).touch()
except:
    pass

In [3]:
con = sqlite3.connect(dbPath)
curs = con.cursor()

## Ingest restaurant Data

In [4]:
rests = pd.read_csv(
    "https://fsa-catalogue2.s3.eu-west-2.amazonaws.com/Approved+establishments+01-09-2022.csv",
    usecols=["TradingName", "Town", "Country", "CompetentAuthority"])


In [5]:
# create the sql table ready for write
curs.execute("""CREATE TABLE IF NOT EXISTS restaurants ('TradingName', 'Town', 'Country', 'CompetentAuthority');""")

## Write CSV to DataBase

In [6]:
# write the csv to the db
rests.to_sql('restaurants', con, if_exists='append', index = False)
del rests

In [7]:
params = ("Wales", "Food Standards Agency")
curs.execute("SELECT TradingName FROM restaurants WHERE Country = ? AND CompetentAuthority = ?", params)
curs.fetchall()

[('2 Sisters Food Group Limited T/A 2 Sisters Llangefini',),
 ('The Food Technology Centre',),
 ('Preseli Bluestone Meats Limited',),
 ('2 Sisters Food Group Limited T/A 2 Sisters Sandycroft',),
 ('Faccenda Foods Limited (T/A Avara Foods)',),
 ('Dunbia (UK)',),
 ('Llechwedd Meats',),
 ('Cig Eryri - Snowdonia Meats Cyf',),
 ("Daphne's Welsh Lamb Limited",),
 ('Celtica Foods Limited',),
 ('Bwydlyn',),
 ('Universal Foods',),
 ('M K (Wales) Ltd T/A (Maddock Kembery Meats)',),
 ('Weeks Wholesale Meat Company Ltd',),
 ('I & B Jones Ltd',),
 ('William Arthur James (T/A WA James Butchers)',),
 ('Maelor Foods Limited',),
 ('Hugh Phillips Gower Butcher',),
 ('Mid Glamorgan Provisions Ltd',),
 ('Farmers Pantry Ltd',),
 ('Williams Lloyd Williams and Son Limited',),
 ('WJ George (Butchers) Ltd',),
 ('Huw Evans & Morag Evans (T/A Cig Oen Caron)',),
 ('Direct Meat Supplies Ltd',),
 ('Conwy Valley Meats Limited',),
 ('David Thomas James Thomas Stephen Thomas & Simeon Thomas (T/A D & J Thomas)',),
 ('C

## Functional Approach

Define a function that can query the database based on parameters received.

In [19]:
def find_restaurants(param_dict, db_path):
    # Create the base query
    baseQuery = 'SELECT * FROM restaurants'
    # ensure params exists
    if len(param_dict) > 0:
        # Add filters for each params value
        param_filters = [f"{k}=?" for k in param_dict]
        filter_query = " WHERE " + " and ".join(param_filters)
    else:
        raise ValueError("params needs an argument.")
    
    # assemble the query
    query = baseQuery + filter_query

    # Create the tuple of values
    t = tuple(param_dict.values())
    # check db is valid
    if os.path.exists(db_path):
        # Open db connection
        conn = sqlite3.connect(db_path)
        # Create cursor
        c = conn.cursor()
        # Execute query
        c.execute(query, t)
        return c.fetchall()
    else:
        raise FileNotFoundError(f"{db_path} not found.")
    return None
    


In [21]:
find_restaurants(
    {"Country":"Wales", "CompetentAuthority":"Food Standards Agency"},
    os.path.join(here(), "data", "restaurants.db")
    )

[('2 Sisters Food Group Limited T/A 2 Sisters Llangefini',
  'Llangefni',
  'Wales',
  'Food Standards Agency'),
 ('The Food Technology Centre', 'Anglesey', 'Wales', 'Food Standards Agency'),
 ('Preseli Bluestone Meats Limited',
  'Carmarthen',
  'Wales',
  'Food Standards Agency'),
 ('2 Sisters Food Group Limited T/A 2 Sisters Sandycroft',
  'Deeside',
  'Wales',
  'Food Standards Agency'),
 ('Faccenda Foods Limited (T/A Avara Foods)',
  'Abergavenny',
  'Wales',
  'Food Standards Agency'),
 ('Dunbia (UK)', 'Llanybydder', 'Wales', 'Food Standards Agency'),
 ('Llechwedd Meats', 'Llangefni', 'Wales', 'Food Standards Agency'),
 ('Cig Eryri - Snowdonia Meats Cyf',
  'Blaenau Ffestiniog',
  'Wales',
  'Food Standards Agency'),
 ("Daphne's Welsh Lamb Limited", 'Denbigh', 'Wales', 'Food Standards Agency'),
 ('Celtica Foods Limited', 'Llanelli', 'Wales', 'Food Standards Agency'),
 ('Bwydlyn', 'Criccieth', 'Wales', 'Food Standards Agency'),
 ('Universal Foods', 'Cardiff', 'Wales', 'Food Standa